In [1]:
import nrrd

In [2]:
# sample image
im_path = 'data/images/LIDC-IDRI-0001.nrrd'

im_V,im_d = nrrd.read(im_path)

In [3]:
im_V.shape

(512, 512, 133)

In [4]:
im_d

OrderedDict([('type', 'int16'),
             ('dimension', 3),
             ('sizes', array([512, 512, 133])),
             ('endian', 'little'),
             ('encoding', 'gzip'),
             ('patient_id', 'LIDC-IDRI-0001'),
             ('slice_thickness', '2.5'),
             ('pixel_spacing', '0.703125'),
             ('contrast_used', 'True')])

In [17]:
from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv
from scipy.ndimage import rotate   # for visualization purposes 

In [6]:
isv(im_V)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [7]:
# sample image
msk_path = 'data/masks/LIDC-IDRI-0001.nrrd'

msk_V,msk_d = nrrd.read(msk_path)

isv(msk_V)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [19]:
from skimage.measure import label,regionprops

# label the mask into connected regions
nodule_labels,num_labels = label(msk_V,return_num=True)

In [39]:
nodule_props = regionprops(nodule_labels)

# the number of nodules in nodule_props corresponds to the number of connected regions
print('There are {} nodule(s) in this image.'.format(len(nodule_props)))        

There are 1 nodule(s) in this image.


In [36]:
# isolate the centroid of the nodule -- this will be our marker for the watershed algorithm
nodule_props[0].centroid
centroid = [round(i) for i in nodule_props[0].centroid]

In [26]:
from skimage.segmentation import watershed

watershed?

In [37]:
import numpy as np

marker_image = np.zeros((msk_V.shape[0],msk_V.shape[1],msk_V.shape[2]),dtype=bool)
marker_image[centroid] = True

In [ ]:
result = watershed(im_V,marker_image,watershed_line=True)

In [ ]:
result.shape
isv(result)